<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-your-start:" data-toc-modified-id="Before-your-start:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before your start:</a></span></li><li><span><a href="#Challenge-1---Independent-Sample-T-tests" data-toc-modified-id="Challenge-1---Independent-Sample-T-tests-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Challenge 1 - Independent Sample T-tests</a></span></li><li><span><a href="#Challenge-2---Matched-Pairs-Test" data-toc-modified-id="Challenge-2---Matched-Pairs-Test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Challenge 2 - Matched Pairs Test</a></span></li><li><span><a href="#Bonus-Challenge---The-Chi-Square-Test" data-toc-modified-id="Bonus-Challenge---The-Chi-Square-Test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Bonus Challenge - The Chi-Square Test</a></span></li></ul></div>

# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas

import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy as sp
import statsmodels.api as sm


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [4]:
# Your code here:

pokemon.Legendary.value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [5]:
# Your code here:

regular_mean = pokemon.loc[pokemon.Legendary == False, 'Total'].mean()
regular_std = pokemon.loc[pokemon.Legendary == False, 'Total'].std()

print(regular_mean, regular_std)

legendary_mean = pokemon.loc[pokemon.Legendary == True, 'Total'].mean()
legendary_std = pokemon.loc[pokemon.Legendary == True, 'Total'].std()

print(legendary_mean, legendary_std)

417.21360544217686 106.76041745713022
637.3846153846154 60.93738905315346


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [6]:
# Your code here:
alpha = 0.05

regular = pokemon.loc[pokemon.Legendary == False, 'Total']
legendary = pokemon.loc[pokemon.Legendary == True, 'Total']

r = sp.stats.ttest_ind(regular, legendary, equal_var=False)
print(r)
print(r.pvalue < alpha)

Ttest_indResult(statistic=-25.8335743895517, pvalue=9.357954335957446e-47)
True


What do you conclude from this test? Write your conclusions below.

In [7]:
# Your conclusions here:

"""Since the pvalue is a lot lower than alpha (0.05), we con clude that the difference between two groups is significant"""

'Since the pvalue is a lot lower than alpha (0.05), we con clude that the difference between two groups is significant'

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [8]:
# Your code here:

pokemon['Type 1'].value_counts()


Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Rock         44
Electric     44
Ground       32
Ghost        32
Dragon       32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [9]:
# Your code here:

water_mean = pokemon.loc[pokemon['Type 1'] == 'Water', 'Total'].mean()
water_std = pokemon.loc[pokemon['Type 1'] == 'Water', 'Total'].std()

print(water_mean, water_std)

other_mean = pokemon.loc[pokemon['Type 1'] != 'Water', 'Total'].mean()
other_std = pokemon.loc[pokemon['Type 1'] != 'Water', 'Total'].std()

print(other_mean, other_std)

print(water_mean>other_mean)
print(water_std>other_std)

430.45535714285717 113.18826606431458
435.85901162790697 121.09168230208066
False
False


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [10]:
# Your code here:

water = pokemon.loc[pokemon['Type 1'] == 'Water', 'Total']
other = pokemon.loc[pokemon['Type 1'] != 'Water', 'Total']

stats.ttest_ind(water, other)

Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [11]:
# Your conclusions here:

"""We are testing that there is a significant difference between the pokemon water and pokemon non-watter with a 95% degree of confidence. Our p-value is higher than alpha (0.05). This means that there is no significant difference between the two means."""


'We are testing that there is a significant difference between the pokemon water and pokemon non-watter with a 95% degree of confidence. Our p-value is higher than alpha (0.05). This means that there is no significant difference between the two means.'

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [12]:
# Your code here:
# H0 = the defense and attack scores are equal

alpha = 0.05
r = stats.ttest_rel(pokemon.Defense, pokemon.Attack)
print(r)
print(r.pvalue < alpha)


Ttest_relResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05)
True


Describe the results of the test in the cell below.

In [13]:
# Your conclusions here:

"""With a p-value much smaller than the alpha (0.05) we reject the null hypothesis. It means that the attack and defense scores are not equal"""

'With a p-value much smaller than the alpha (0.05) we reject the null hypothesis. It means that the attack and defense scores are not equal'

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [14]:
# Your code here:

pokemon.head()

alpha = 0.05
r1 = stats.ttest_rel(pokemon['Sp. Def'], pokemon['Sp. Atk'])
print(r1)
print(r1.pvalue < alpha)


Ttest_relResult(statistic=-0.853986188453353, pvalue=0.3933685997548122)
False


Describe the results of the test in the cell below.

In [15]:
# Your conclusions here:

"""We are testing that there is a significant difference between the mean of special defense and the mean of special attack with a 95% degree of confidence. Our p-value is higher than alpha (0.05). This means that there is no significant difference between the two means."""


'We are testing that there is a significant difference between the mean of special defense and the mean of special attack with a 95% degree of confidence. Our p-value is higher than alpha (0.05). This means that there is no significant difference between the two means.'

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [16]:
# Your code here:
# H0=the difference between the means is zero
alpha = 0.05
r2 = stats.ttest_1samp(pokemon.Defense-pokemon.Attack, 0)
print(r2)
print(r2.pvalue < alpha)

print(r2==r)

Ttest_1sampResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05)
True
True


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [17]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [18]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [19]:
# Your answer here:

